In [ ]:
import cv2
import winsound
cam = cv2.VideoCapture(0)
while cam.isOpened():
    ret, frame1 = cam.read()
    ret, frame2 = cam.read()
#     diff = cv2.absdiff(frame1 , frame2)
#     gray = cv2.cvtColor(diff, cv2.COLOR_RGB2GRAY)
#     blur = cv2.GaussianBlur(gray,(5,5),0)
#     _, thresh = cv2.threshold(blur,20,255,cv2.THRESH_BINARY)
#     dilated = cv2.dilate(thresh,None , iterations=3)
#     contours, _ = cv2.findContours(dilated, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#     #cv2.drawContours(frame1,contours, -1, (0,255,0),2)
#     for c in contours:
#         if cv2.contourArea(c) < 5000:
#             continue
#         x , y , w, h = cv2.boundingRect(c)
#         cv2.rectangle(frame1 , (x,y), (x+w,y+h),(0,255,0),2)
#         winsound.PlaySound('alert.wav',winsound.SND_ASYNC)
#     print(frame1.shape)
    if cv2.waitKey(10) == ord('q'):
        break
    cv2.imshow('Raj Cam', frame1)
cv2.destroyAllWindows()

In [11]:
#image
import cv2
import numpy as np
import time

# def code():
cam = cv2.VideoCapture(0)
# print("[INFO] loading YOLO from disk...")
weights_path = '.\\yolov3-tiny.weights'
cfg_path = '.\\yolov3-tiny.cfg'

while cam.isOpened():
    global weights_path
    global cfg_path
    
    ret, image = cam.read()
    
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
    
    neuralNet = cv2.dnn.readNetFromDarknet(cfg_path, weights_path)

    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),swapRB=True, crop=False)

    neuralNet.setInput(blob)

    layer_names = neuralNet.getLayerNames()

    last_layer_indices = neuralNet.getUnconnectedOutLayers()

    last_layer_names = [ layer_names[index[0]-1] for index in last_layer_indices] # 35, 47
    
    start = time.time()
    layerOutputs = neuralNet.forward(last_layer_names)
    end = time.time()
    
#     print("[INFO] YOLO took {:.6f} seconds".format(end - start))
    rectangles = []
    confidences = []
    
    H,W = image.shape[:2]
    
    for outputs in layerOutputs:
        for box in outputs:
            dimensions = box[:4]
            scores = box[5:]
            classID = np.argmax(scores)
            if classID != 0:
                continue

            confidence = scores[classID]

            if confidence > 0.5:
                rect = dimensions * np.array([W, H, W, H])
                centerX, centerY, width, height = rect.astype("int")

                startX = int(centerX - width/2)
                startY = int(centerY - height/2)
                rectangles.append([startX, startY, int(width),int(height)])
                confidences.append(float(confidence))
#     print(len(rectangles))
    idxs = cv2.dnn.NMSBoxes(rectangles, confidences, 0.5,0.5)    
#     print(idxs)
    for idx in idxs:
        idx = idx[0]
        startX, startY, width, height = rectangles[idx]
        endX = startX + width
        endY = startY + height
        cv2.rectangle(image, (startX, startY), (endX, endY), (255,0,0), 2)

    if cv2.waitKey(10) == ord('q'):
        cv2.destroyAllWindows()
        break

    cv2.imshow('image',image)

    


# code()    

# 1 Frame - 1.884941 seconds

In [ ]:
import os
import cv2
import numpy as np
import time
# derive the paths to the YOLO weights and model configuration
weightsPath =  "yolov3.weights"
configPath = "yolov3.cfg"
# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)


# load our input image and grab its spatial dimensions
image = cv2.imread('person3.jpg')
(H, W) = image.shape[:2]
# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
# construct a blob from the input image and then perform a forward
# pass of the YOLO object detector, giving us our bounding boxes and
# associated probabilities
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
	swapRB=True, crop=False)
net.setInput(blob)
start = time.time()
layerOutputs = net.forward(ln)
end = time.time()
# show timing information on YOLO
print("[INFO] YOLO took {:.6f} seconds".format(end - start))
# class IDs, respectively
boxes = []
confidences = []
classIDs = []
LABELS = []
with open('labels.txt') as file:
    LABELS = [x.strip("\n") for x in file.readlines()]

for output in layerOutputs:
    # loop over each of the detections
    for detection in output:
        # extract the class ID and confidence (i.e., probability) of
        # the current object detection
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        
#         print(confidence)
        # probability is greater than the minimum probability
        if confidence > 0.5:
            # scale the bounding box coordinates back relative to the
            # size of the image, keeping in mind that YOLO actually
            # returns the center (x, y)-coordinates of the bounding
            # box followed by the boxes' width and height
            box = detection[0:4] * np.array([W, H, W, H])
            (centerX, centerY, width, height) = box.astype("int")
            # use the center (x, y)-coordinates to derive the top and
            # and left corner of the bounding box
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))
            # update our list of bounding box coordinates, confidences,
            # and class IDs
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            classIDs.append(classID)
idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,0.5)
print(idxs)
# ensure at least one detection exists
if len(idxs) > 0:
    # loop over the indexes we are keeping
    for i in idxs.flatten():
        # extract the bounding box coordinates
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])
        # draw a bounding box rectangle and label on the image
        cv2.rectangle(image, (x, y), (x + w, y + h), (255,0,0), 2)
        text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
        cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
        0.5, (255,0,0), 2)
# show the output image
# cv2.imshow("Image", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
layerOutputs[0][0] 
3.9945053e-08